## SPARC-4 mini-pipeline tools
***
# <font color='darkblue'>Polarimetry</font> 

This notebook shows an example for obtaining a polarimetric analysis of a set of observations using the SPARC4 instrument in polarimetric mode with both half-wave and quarter-wave rotating plates. It takes as input a series of science imaging products `*_proc.fits` obtained with the SPARC4 pipeline.

* Eder Martioli, LNA 20 Jun 2023

<div class="alert alert-block alert-warning">
<b>WARNING:</b> it requires running <b>sp4_sci_image_reduction.ipynb</b> in advance, for the same channel and modes, to generate *_proc.fits reduced files
</div>

In [ ]:
import sparc4.product_plots as s4plt
import sparc4.pipeline_lib as s4pipelib
import sparc4.utils as s4utils
import sparc4.db as s4db

## User inputs

In [ ]:
# set an object ID matching the ID in the image header keyword 'OBJECT'
OBJECTID = "HD111579"

# set night directory name
NIGHTDIR = '20230503'
# set raw data root directory
DATADIR =  "/Volumes/Samsung_T5/Data/SPARC4/minidata/"
# set reduced data root directory
REDUCEDDIR = "/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/"

# set SPARC4 channel
CHANNEL = 4  # 1, 2, 3 or 4

# whether or not to force reduction even if product already exists
FORCE = False

# get SPARC4 pipeline parameters
p = s4pipelib.init_s4_p(datadir=DATADIR,
                        reducedir=REDUCEDDIR,
                        nightdir=NIGHTDIR,
                        channels="{}".format(CHANNEL),
                        print_report=False)

# create database of raw data for reduction
db = s4db.create_db_from_observations(p['filelists'][CHANNEL-1], 
                                      p['DB_KEYS'], 
                                      include_img_statistics=p["INCLUDE_IMG_STATISTICS"], 
                                      include_only_fullframe=p["FULL_FRAMES_ONLY"], 
                                      output=p['s4db_files'][CHANNEL-1])

# detect all detector modes
detector_modes = s4db.get_detector_modes_observed(db, 
                                                  science_only=True,
                                                  detector_keys=p["DETECTOR_MODE_KEYWORDS"])
# get first valid key
mode_key = next(iter(detector_modes))

# set PHOTOMETRIC mode
inst_mode = p['INSTMODE_POLARIMETRY_KEYVALUE']

# we need the reduce directory 
reduce_ch_dir = p['reduce_directories'][CHANNEL-1]

# 1. Calculate `HALF-WAVE` POLARIMETRY

In [ ]:
# First set the polar mode and waveplate to "halfwave"
polar_mode = p['POLARIMETRY_L2_KEYVALUE']
polsuffix = "_{}_{}".format(inst_mode, polar_mode)

# get list of objects observed in polarimetric mode
objs = s4db.get_targets_observed(db, 
                                 inst_mode=inst_mode, 
                                 polar_mode=polar_mode, 
                                 detector_mode=detector_modes[mode_key])

# select object's data 
if OBJECTID != "all" :
    objs = objs[objs['OBJECT'] == OBJECTID]

    
# loop over each object to run the reduction
for k in range(len(objs)) :
    obj = objs[k][0]
    
    # set suffix for output time series filename
    ts_suffix = "{}_s4c{}_{}{}".format(NIGHTDIR,
                                     p['CHANNELS'][CHANNEL-1] ,
                                     obj.replace(" ",""), 
                                     polsuffix)    
    
    # get list of science data matching object, detector mode, inst. mode, and polar mode
    raw_sci_list = s4db.get_file_list(db,
                                      object_id=obj, 
                                      inst_mode=inst_mode, 
                                      polar_mode=polar_mode, 
                                      obstype=p['OBJECT_OBSTYPE_KEYVALUE'], 
                                      calwheel_mode=None, 
                                      detector_mode=detector_modes[mode_key])
    # input reduced files list
    sci_list = ["{}/{}".format(reduce_ch_dir, os.path.basename(f).replace(".fits","_proc.fits")) for f in raw_sci_list]
    
    # group input list into polarimetric sequences
    pol_sequences = s4utils.select_polar_sequences(sci_list, sortlist=True, verbose=True)
    
    for i in range(len(pol_sequences)) :    
        
        if len(pol_sequences[i]) == 0 :
            continue
            
        polarproduct = s4pipelib.compute_polarimetry(pol_sequences[i],
                                                     wave_plate = 'halfwave',
                                                     base_aperture = p['APERTURE_RADIUS_FOR_PHOTOMETRY_IN_POLAR'],
                                                     compute_k = True,
                                                     fit_zero = False,
                                                     zero = 0)

        pol_results = s4pipelib.get_polarimetry_results(polarproduct,
                                                        source_index=0,
                                                        min_aperture=p['MIN_APERTURE_FOR_POLARIMETRY'],
                                                        max_aperture=p['MAX_APERTURE_FOR_POLARIMETRY'],
                                                        plot=True,
                                                        verbose=True)

# 2. Calculate `QUARTER-WAVE` POLARIMETRY

In [ ]:
# First set the polar mode and waveplate to "quarterwave"
polar_mode = p['POLARIMETRY_L4_KEYVALUE']
polsuffix = "_{}_{}".format(inst_mode, polar_mode)

# get list of objects observed in polarimetric mode
objs = s4db.get_targets_observed(db, 
                                 inst_mode=inst_mode, 
                                 polar_mode=polar_mode, 
                                 detector_mode=detector_modes[mode_key])

# select object's data 
if OBJECTID != "all" :
    objs = objs[objs['OBJECT'] == OBJECTID]

    
# loop over each object to run the reduction
for k in range(len(objs)) :
    obj = objs[k][0]
    
    # set suffix for output time series filename
    ts_suffix = "{}_s4c{}_{}{}".format(NIGHTDIR,
                                     p['CHANNELS'][CHANNEL-1] ,
                                     obj.replace(" ",""), 
                                     polsuffix)    
    
    # get list of science data matching object, detector mode, inst. mode, and polar mode
    raw_sci_list = s4db.get_file_list(db,
                                      object_id=obj, 
                                      inst_mode=inst_mode, 
                                      polar_mode=polar_mode, 
                                      obstype=p['OBJECT_OBSTYPE_KEYVALUE'], 
                                      calwheel_mode=None, 
                                      detector_mode=detector_modes[mode_key])
    # input reduced files list
    sci_list = ["{}/{}".format(reduce_ch_dir, os.path.basename(f).replace(".fits","_proc.fits")) for f in raw_sci_list]
    
    # group input list into polarimetric sequences
    pol_sequences = s4utils.select_polar_sequences(sci_list, sortlist=True, verbose=True)
    
    for i in range(len(pol_sequences)) :    
        
        if len(pol_sequences[i]) == 0 :
            continue
            
        polarproduct = s4pipelib.compute_polarimetry(pol_sequences[i],
                                                     wave_plate = 'quarterwave',
                                                     base_aperture = p['APERTURE_RADIUS_FOR_PHOTOMETRY_IN_POLAR'],
                                                     compute_k = False,
                                                     fit_zero = True,
                                                     zero = None)

        pol_results = s4pipelib.get_polarimetry_results(polarproduct,
                                                        source_index=0,
                                                        min_aperture=p['MIN_APERTURE_FOR_POLARIMETRY'],
                                                        max_aperture=p['MAX_APERTURE_FOR_POLARIMETRY'],
                                                        plot=True,
                                                        verbose=True)